In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Load the dataset
data = pd.read_csv('/kaggle/input/australia-weather-data/Weather Training Data.csv')

# Separate features (X) and target variable (y)
X = data.drop(columns=['RainTomorrow'])  # Features: all columns except 'RainTomorrow'
y = data['RainTomorrow']  # Target variable: 'RainTomorrow'

# Convert categorical variables to dummy/indicator variables
X = pd.get_dummies(X)

# Handle missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')  # You can use different strategies as needed
X_imputed = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Load the dataset
data = pd.read_csv('/kaggle/input/australia-weather-data/Weather Training Data.csv')

# Handle missing values
data.dropna(inplace=True)

# Prepare features (X) and target variable (y)
X = data.drop(columns=['RainToday'])  # Features
y = data['RainToday']  # Target variable

# Convert categorical variables to numeric using label encoding
label_encoder = LabelEncoder()
X['WindGustDir'] = label_encoder.fit_transform(X['WindGustDir'])
X['WindDir9am'] = label_encoder.fit_transform(X['WindDir9am'])
X['WindDir3pm'] = label_encoder.fit_transform(X['WindDir3pm'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Initialize the model
model = RandomForestClassifier()

# Train the model
model.fit(X_train_imputed, y_train)

# Make predictions
predictions = model.predict(X_test_imputed)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier

# Load the dataset
data = pd.read_csv("/kaggle/input/australia-weather-data/Weather Training Data.csv")  # Replace with the actual file path

# Drop columns not needed for prediction
data = data.drop(columns=['Evaporation', 'Sunshine'])

# Remove rows with missing values
data = data.dropna()

# Encode categorical variables
le = LabelEncoder()
data['Location'] = le.fit_transform(data['Location'])
data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
data['WindDir9am'] = le.fit_transform(data['WindDir9am'])
data['WindDir3pm'] = le.fit_transform(data['WindDir3pm'])
data['RainToday'] = le.fit_transform(data['RainToday'])

# Select features and target variable
X = data[['WindSpeed3pm', 'WindSpeed9am', 'WindGustDir', 'WindDir9am', 'WindDir3pm']]
y = data['RainToday']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define base models
base_models = [
     ("AdaBoost", AdaBoostClassifier(random_state=42)),
    ("LGBM", LGBMClassifier(random_state=42)),
    ("MLP", MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)),
    ("XGBoost", XGBClassifier(random_state=42)),
     ("SVM", SVC(random_state=42)),
#     ("Logistic Regression", LogisticRegression(random_state=42))
]

# Create the ensemble model
ensemble_model = VotingClassifier(estimators=base_models, voting='hard')

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Evaluate the ensemble model
y_pred = ensemble_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))